# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [33]:
df = pd.read_csv("hasil_clustering.csv")

In [34]:
df.head()

,Unit_Price,Quantity,Discount,Total_Price,Customer_Type,Cluster
0,1.66,53.0,0.10,79.18,B2B,1.0
1,1.56,90.0,0.10,126.36,B2B,1.0
2,1.17,73.0,0.05,81.14,B2B,1.0
3,3.22,59.0,0.10,170.98,B2B,1.0
4,0.87,35.0,0.10,27.40,B2B,1.0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [35]:
from sklearn.impute import SimpleImputer

# 1. Pisahkan fitur (X) dan label (y)
X = df.drop(columns=['Cluster'])
y = df['Cluster']

In [36]:
# 2. Encoding fitur kategorik
X_encoded = X.copy()
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col])

In [37]:
# 3. Imputasi untuk missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_encoded)

In [38]:
# 4. Normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [42]:
mask = y.notna()
X_scaled_clean = X_scaled[mask]
y_clean = y[mask]

In [45]:
# 5. Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled_clean, y_clean, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

X_train shape: (165161, 5), X_test shape: (41291, 5)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [46]:
# KNN dan Random Forest Training
knn = KNeighborsClassifier().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)

K-Nearest Neighbors (KNN) merupakan algoritma klasifikasi berbasis instance-based learning. Pada KNN, proses klasifikasi dilakukan dengan mengukur jarak antara data baru dengan seluruh data dalam dataset pelatihan. Kemudian, algoritma ini menentukan kelas dari data baru berdasarkan mayoritas kelas dari sejumlah k tetangga terdekatnya. KNN memiliki keunggulan berupa kesederhanaan dalam implementasi serta performa yang cukup baik pada dataset berukuran kecil hingga menengah. Namun, KNN juga memiliki kelemahan, yaitu sensitif terhadap skala data sehingga sebelum digunakan, data perlu melalui proses normalisasi atau standarisasi. Selain itu, KNN dapat menjadi lambat saat digunakan untuk memprediksi data dalam jumlah besar karena perlu menghitung jarak ke seluruh data pelatihan setiap kali prediksi dilakukan.


Di sisi lain, algoritma Random Forest digunakan sebagai alternatif yang lebih kompleks dan robust. Random Forest merupakan metode ensemble yang membangun banyak pohon keputusan (decision tree) selama proses pelatihan dan menggabungkan hasil prediksi dari masing-masing pohon untuk menghasilkan keputusan akhir. Dalam pembentukannya, Random Forest menggunakan teknik bootstrap aggregating (bagging), di mana subset acak dari data dan fitur digunakan untuk membangun setiap pohon, sehingga model menjadi lebih tahan terhadap overfitting dibandingkan dengan decision tree tunggal. Keunggulan utama dari Random Forest adalah kemampuannya untuk menangani dataset yang besar dan kompleks, serta memberikan estimasi pentingnya setiap fitur dalam proses klasifikasi. Namun, kekurangannya adalah interpretasi hasil model yang lebih sulit dan kebutuhan sumber daya komputasi yang lebih besar, baik saat pelatihan maupun saat prediksi.

Pemilihan kedua algoritma ini bertujuan untuk membandingkan kinerja model sederhana seperti KNN dengan model ensemble seperti Random Forest, sehingga diperoleh gambaran yang lebih komprehensif terhadap data yang digunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [47]:
# ======== KNN ========
y_pred_knn = knn.predict(X_test)

print("=== K-Nearest Neighbors (KNN) ===")
print("Accuracy :", accuracy_score(y_test, y_pred_knn))
print("Precision:", precision_score(y_test, y_pred_knn, average='weighted'))
print("Recall   :", recall_score(y_test, y_pred_knn, average='weighted'))
print("F1 Score :", f1_score(y_test, y_pred_knn, average='weighted'))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_knn))

=== K-Nearest Neighbors (KNN) ===
Accuracy : 0.9998304715313264
Precision: 0.9998308082116879
Recall   : 0.9998304715313264
F1 Score : 0.9998305841437325
Confusion Matrix:
[[26119     0     0]
 [    0 14123     5]
 [    0     2  1042]]


In [48]:
# ======== Random Forest ========
y_pred_rf = rf.predict(X_test)

print("=== Random Forest ===")
print("Accuracy :", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf, average='weighted'))
print("Recall   :", recall_score(y_test, y_pred_rf, average='weighted'))
print("F1 Score :", f1_score(y_test, y_pred_rf, average='weighted'))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

=== Random Forest ===
Accuracy : 0.9998789082366617
Precision: 0.9998788669391323
Recall   : 0.9998789082366617
F1 Score : 0.999878881368384
Confusion Matrix:
[[26119     0     0]
 [    0 14126     2]
 [    0     3  1041]]


In [49]:

# Buat data evaluasi
evaluation_results = {
    'Model': ['K-Nearest Neighbors', 'Random Forest'],
    'Accuracy': [0.9998, 0.9999],
    'Precision': [0.9998, 0.9999],
    'Recall': [0.9998, 0.9999],
    'F1 Score': [0.9998, 0.9999]
}

# Buat DataFrame
df_evaluation = pd.DataFrame(evaluation_results)

# Tampilkan
print("Tabel Hasil Evaluasi Model:")
display(df_evaluation)

Tabel Hasil Evaluasi Model:


,Model,Accuracy,Precision,Recall,F1 Score
0,K-Nearest Neighbors,0.9998,0.9998,0.9998,0.9998
1,Random Forest,0.9999,0.9999,0.9999,0.9999


## Evaluasi Model Klasifikasi: K-Nearest Neighbors (KNN) dan Random Forest

- **KNN**:

   Accuracy : 99.98%

   Precision: 99.98%

   Recall : 99.98%

   F1 Score : 99.98%

   Analisis : Model KNN menunjukkan performa yang sangat tinggi dengan akurasi hampir sempurna. Kesalahan klasifikasi sangat kecil, hanya terjadi pada beberapa sampel di cluster 2 dan 3. Ini menunjukkan bahwa KNN dapat memetakan pola distribusi data dengan baik.

- **Random Forest**:

   Accuracy : 99.99%

   Precision: 99.99%

   Recall : 99.99%

   F1 Score : 99.99%

   Analisis : Random Forest juga memberikan hasil yang sangat baik, bahkan sedikit lebih unggul dibandingkan KNN dalam hal akurasi dan jumlah kesalahan klasifikasi yang lebih sedikit. Random Forest mampu mengklasifikasikan data dengan lebih stabil berkat teknik ensemble yang menggabungkan banyak decision tree.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Hyperparameter tuning tidak dilakukan karena model telah menghasilkan performa yang sangat tinggi (>99%) baik pada K-Nearest Neighbors (KNN) maupun Random Forest.


**Identifikasi Kelemahan Model:**
- Dari confusion matrix, terlihat bahwa model hampir tidak membuat kesalahan prediksi.

- Error kecil yang muncul hanya pada beberapa sample di kelas minoritas.

- Tidak ada indikasi Precision atau Recall yang rendah untuk kelas tertentu, karena nilai semua metrik sangat tinggi (>99%).

- Model tidak menunjukkan tanda-tanda overfitting maupun underfitting:

  - Hasil evaluasi pada data testing tetap tinggi.

  - Tidak ada gap besar antara prediksi dan aktual.

**Rekomendasi Tindakan Lanjutan:**
- Monitoring performa secara berkala bila diterapkan pada data baru di dunia nyata.

- Jika nantinya akurasi menurun saat deployment:

- Pertimbangkan melakukan hyperparameter tuning pada KNN (misal memilih n_neighbors terbaik) atau Random Forest (n_estimators, max_depth).

- Bisa juga mencoba algoritma lain seperti XGBoost atau LightGBM untuk mengejar performa lebih optimal.

- Mengumpulkan data tambahan disarankan untuk menjaga generalisasi model ke depannya.